In [80]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import os
import re
import glob
from pypion.ReadData import ReadData
import astropy.units as u

import pathlib
home = str(pathlib.Path.home())


### Info

Simulation started at covertex before periastron

This coincides with a time in seconds before periastron of:

$$2.679 \times 10^7 s$$

In [125]:
start_time = -2.679e7 * u.s
print(start_time.to(u.yr))

-0.8489238725378355 yr


In [98]:
def make_snapshots(data_path):
        ########## Cataloging silo files ###############################
        # Get the list of silo files
        silo_files = os.listdir(data_path)
        silo_files = sorted([f for f in silo_files if f.endswith('.silo')])
        filename=(silo_files[0]).replace('_level00_0000.00000000.silo','')
        
        print("Info from silo files:")
        print(f"Basename of silo files: {filename}")

        file_list = glob.glob(os.path.join(data_path, '*.silo'), recursive=False)

        level_list = []
        files = []

        for file in file_list:
            level = re.search('_level(.*)_', file)
            if level == None:
                pass
            else:
                level = level.group(1)
                if not level in level_list:
                    level_list.append(level)
        level_list.sort()

        ########## Categorizing data files into levels #################
        if len(level_list) == 1: 
            print('Simulation Info: Single level')
            catalog = []
            files = sorted(glob.glob(filename + '_0000.*.silo'))
            catalog.append(files)
        else:
            print(f'Simulation Info: {len(level_list)} levels')
            catalog = []
            for i in range(len(level_list)):
                files = sorted(glob.glob(os.path.join(data_path, f"{filename}_level{level_list[i]}_0000.*.silo")))
                catalog.append(files)
                

        # Bundle silo files of different levels of same time instant into a snapshot.
        evolution = np.array(catalog).T
        print(f"Number of snapshots: {evolution.shape[0]}")

        return evolution

In [99]:
data_dir = "/mnt/massive-stars/data/thomas_simulations/wr140-sims/covertex_start/red_z_res/wr140-mhd-l7n256/"

evolution = make_snapshots(data_dir)

fits_dir = os.path.join(data_dir, "proj/XY_proj2")
png_dir = os.path.join(home, "code/project/images/xray-proj/wr140-mhd-n256/XY/")

if not os.path.exists(png_dir):
    os.makedirs(png_dir)

Info from silo files:
Basename of silo files: wr140_mhd_cool_d3l7n256
Simulation Info: 7 levels
Number of snapshots: 285


In [201]:
def create_xray_proj_img(data_path, fits_dir, png_dir, start_time, **kwargs):
    start_time = start_time*u.s

    evolution = make_snapshots(data_path)

    fits_dir = os.path.join(data_path, fits_dir)
    png_dir = os.path.join(home, png_dir)

    if not os.path.exists(png_dir):
        os.makedirs(png_dir)

    fits_files = sorted(os.listdir(fits_dir))
    fits_files.sort(key=lambda test_string : list(
        map(int, re.findall(r'\d+', test_string))))

    def snapshots_before_after_periastron(evolution, tolerance, start_time):
        sim_time_array = []
        for file in evolution:
            data = ReadData(file)
            sim_time = data.sim_time().to(u.yr) + start_time.to(u.yr)
            data.close()
            sim_time_array.append(np.abs(sim_time.value))
        
        sim_time_array = np.array(sim_time_array)
        args = np.where(sim_time_array < tolerance)

        return list(args[0])
    
    args = snapshots_before_after_periastron(evolution, kwargs.get("timerange", 0.01), start_time)

    for i in args:
        file = fits_files[i]
        data = ReadData(evolution[i])
        sim_time = data.sim_time().to(u.yr) + start_time.to(u.yr)

        data.close()
        print(f"Processing {file} at {sim_time:3e}")

        hdul = fits.open(os.path.join(fits_dir, file))
        data = hdul[9].data - hdul[11].data
        hdul.close()

        fig, ax = plt.subplots(figsize=(10,10))
        image = ax.imshow(data, cmap="viridis", origin="lower", vmin=0, vmax=0.00225)
        fig.colorbar

        cb = fig.colorbar(image, ax=ax, orientation="vertical", pad=0.1)

        if sim_time < 0:
            ax.text(0.05, 0.95, f"t = {abs(sim_time):.4f} before periastron", 
            bbox=dict(facecolor='white', alpha=0.5), transform=ax.transAxes)
        else:
            ax.text(0.05, 0.95, f"t = {sim_time:.4f} after periastron", 
            bbox=dict(facecolor='white', alpha=0.5), transform=ax.transAxes)

        plt.tick_params(axis='both', bottom=False, left=False, labelbottom=False, labelleft=False)
        plt.savefig(os.path.join(png_dir, file[:-5] + ".png"), bbox_inches="tight", pad_inches=0)
        plt.close(fig)
    




In [202]:
create_xray_proj_img(data_dir, "proj/XY_proj2", "code/project/images/xray-proj/wr140-mhd-n256/XY/", start_time.value, timerange=0.04)

Info from silo files:
Basename of silo files: wr140_mhd_cool_d3l7n256
Simulation Info: 7 levels
Number of snapshots: 285
Processing wr140_mhd_cool_d3l7n256_XY_img.00440320.fits at -3.737547e-02 yr
Processing wr140_mhd_cool_d3l7n256_XY_img.00442368.fits at -3.302111e-02 yr
Processing wr140_mhd_cool_d3l7n256_XY_img.00444416.fits at -2.865863e-02 yr
Processing wr140_mhd_cool_d3l7n256_XY_img.00446464.fits at -2.428196e-02 yr
Processing wr140_mhd_cool_d3l7n256_XY_img.00448512.fits at -1.988378e-02 yr
Processing wr140_mhd_cool_d3l7n256_XY_img.00450560.fits at -1.547994e-02 yr
Processing wr140_mhd_cool_d3l7n256_XY_img.00452608.fits at -1.105682e-02 yr
Processing wr140_mhd_cool_d3l7n256_XY_img.00454656.fits at -6.643551e-03 yr
Processing wr140_mhd_cool_d3l7n256_XY_img.00456704.fits at -2.241291e-03 yr
Processing wr140_mhd_cool_d3l7n256_XY_img.00458752.fits at 2.152564e-03 yr
Processing wr140_mhd_cool_d3l7n256_XY_img.00460800.fits at 6.532131e-03 yr
Processing wr140_mhd_cool_d3l7n256_XY_img.004

In [152]:
# fits_files = sorted(os.listdir(fits_dir))
# fits_files.sort(key=lambda test_string : list(
#         map(int, re.findall(r'\d+', test_string))))

In [153]:
# def snapshots_before_after_periastron(evolution, tolerance, start_time):
    
#     sim_time_array = []
#     for file in evolution:
#         data = ReadData(file)
#         sim_time = data.sim_time().to(u.yr) + start_time.to(u.yr)
#         data.close()
#         sim_time_array.append(np.abs(sim_time.value))
    
#     sim_time_array = np.array(sim_time_array)
#     args = np.where(sim_time_array < tolerance)

#     return list(args[0])

    

In [154]:
# args = snapshots_before_after_periastron(evolution, 0.03, start_time)
# print(args)

In [155]:
# # i = args[0]
# # for file in fits_files[args[0]+1:args[-1]+2]:

# i = 0
# for file in fits_files:
#     data = ReadData(evolution[i])
#     sim_time = data.sim_time().to(u.yr) + start_time.to(u.yr)
#     data.close()
#     i += 1

#     hdul = fits.open(os.path.join(fits_dir, file))
#     data = hdul[9].data -

#     fig, ax = plt.subplots(figsize=(10,10))
#     ax.grid(True, color="white")
#     ax.imshow(data, cmap="inferno")
#     ax.text(0.05, 0.95, f"t = {abs(sim_time):.4f} before periastron", bbox=dict(facecolor='white', alpha=0.5), transform=ax.transAxes)
#     plt.tick_params(axis='both', bottom=False, left=False, labelbottom=False, labelleft=False)
#     plt.savefig(os.path.join(png_dir, file[:-4] + "png"), bbox_inches="tight", pad_inches=0)
#     plt.close()

In [156]:
# # hdul = fits.open('/mnt/massive-stars/data/thomas_simulations/wr140-sims/covertex_start/red_z_res/wr140-hydro-l7n128/proj/wr140_hydro_cool_d3l7n128_XZ_img.00126976.fits')
# # hdul = fits.open("/mnt/massive-stars/data/thomas_simulations/wr140-sims/covertex_start/red_z_res/wr140-mhd-l7n256/proj/wr140_mhd_cool_d3l7n256_XY_img.00526336.fits")
# hdul = fits.open("/mnt/massive-stars/data/thomas_simulations/wr140-sims/covertex_start/red_z_res/wr140-mhd-l7n256/proj/XY_proj/wr140_mhd_cool_d3l7n256_XY_img.00475136.fits")

# hdul.info()

In [157]:
# """ Display the image data with Matplotlib """

# plt.figure()
# plt.imshow(hdul[9].data-hdul[10].data, cmap='inferno')
# plt.colorbar(location='right', label = 'Intensity')
# plt.xlim(274, 1024-274)
# plt.ylim(1024-274, 274)
# plt.tick_params(axis='both', bottom=False, left=False, labelbottom=False, labelleft=False)
# plt.show()